# Jedi Council sentiments

### "Fear is the path to the dark side... fear leads to anger... anger leads to hate.. hate leads to suffering."

![picture](./images/jedi-council.png)

Vamos a analizar con nuestra API el sentimiento de este famoso diálogo del Episodio I. Importamos brevemente el dataset y nos quedamos con las frases que nos interesan.

In [1]:
import pandas as pd
yoda = pd.read_csv('inputs/yoda-dialogues.csv')
yoda_filt = yoda[(yoda['character'] != 'narrator') & (yoda['scene']==134)][['character','text']]
yoda_filt

,character,text
28,ANAKIN,A ship...a cup...a speeder.
30,YODA,"Good, good, young one. How feel you?"
31,ANAKIN,"Cold, sir."
32,YODA,Afraid are you?
33,ANAKIN,"No, sir."
34,MACE WINDU,Afraid to give up your life?
35,ANAKIN,I don't think so.
37,YODA,"See through you, we can."
38,MACE WINDU,Be mindful of your feelings...
39,KI-ADI,YOur thoughts dwell on your mother.


### Ahora insertamos el dataset en MongoDB con llamadas a la API

In [2]:
import requests

Insertamos a los personajes que participan

In [3]:
jedis = list(yoda_filt['character'].unique())
for jedi in jedis:
    res = requests.get(f'http://localhost:5000/user/create/{jedi}').text
    print(res)

Usuario ANAKIN añadido con id 12
Usuario YODA añadido con id 13
Usuario MACE WINDU añadido con id 14
Usuario KI-ADI añadido con id 15


Creamos un grupo con ellos

In [4]:
data = {"nombre":"Jedi Council, Ep I",
        "usuarios":jedis}
resultado = requests.get('http://localhost:5000/chat/create', params=data).text
print(resultado)

Chat 1003 creado correctamente


In [5]:
chatid = resultado[5:9]

Insertamos los mensajes

In [6]:
for a, b in zip(yoda_filt['character'], yoda_filt['text']):
    enviado = requests.get(f'http://localhost:5000/chat/{chatid}/addmessage', params={"usuario":a,"mensaje":b}).text
    print(enviado)

✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado
✓✓ Mensaje enviado


Analizamos el sentimiento del diálogo, que como vemos sale más negativo que positivo:

In [7]:
response = requests.get(f'http://localhost:5000/chat/{chatid}/sentiment').json()
response

{'compound': -0.9615, 'neg': 0.222, 'neu': 0.703, 'pos': 0.075}

From the documentation:
- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
- negative sentiment: compound score <= -0.05

In [8]:
recom = requests.get('http://localhost:5000/user/YODA/recommend').json()
recom

{'ANAKIN': 0.161, 'KI-ADI': 0.0, 'MACE WINDU': 0.173}